## set up

In [2]:
# imports
import os
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
import json
from pydantic import BaseModel, Field
import string
import re
from typing import Optional 
import asyncio
import aiofiles

/Users/catherineyeh/.local/share/virtualenvs/story-viz-mNgXibik/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Define a semaphore to limit concurrency
SEMAPHORE_LIMIT = 20
semaphore = asyncio.Semaphore(SEMAPHORE_LIMIT)

async def semaphore_wrapper(func, *args):
    """Wrap async function with semaphore to limit concurrency"""
    async with semaphore:
        return await func(*args)

In [4]:
# api keys
try:
    with open("../secrets.json") as f:
        secrets = json.load(f)
    open_ai_key = secrets["openai"]
    anthropic_key = secrets["anthropic"]
    google_key = secrets["google"]
    os.environ["OPENAI_API_KEY"] = open_ai_key
    os.environ["ANTHROPIC_API_KEY"] = anthropic_key
    os.environ["GOOGLE_API_KEY"] = google_key
    print("API key loaded.")
except FileNotFoundError:
    print("Secrets file not found. YOU NEED THEM TO RUN THIS.")

API key loaded.


In [5]:
## make sure all folders are created inside notebooks folder
if not os.path.exists("scripts"):
    os.makedirs("scripts")
if not os.path.exists("json"):
    os.makedirs("json")
if not os.path.exists("chapters"):
    os.makedirs("chapters")

In [6]:
# NOT VERY RELEVANT
def format_gemini_response(res):
    output_formatted = res.content[7:-3]
    # turn into json
    output = json.loads(output_formatted)
    return output

class DotDict:
    # allow dot notation for dicts
    def __init__(self, dictionary):
        # check if dictionary is a string
        if isinstance(dictionary, str):
            # if it is a string, convert it to a dictionary
            print(dictionary)
            dictionary = json.loads(dictionary)
        for key, value in dictionary.items():
            setattr(self, key, value)

In [7]:
model_type = "default"
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
# llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
llm2 = ChatAnthropic(model_name="claude-3-5-sonnet-20240620", temperature=0.1, max_tokens_to_sample=4096, api_key=os.getenv("ANTHROPIC_API_KEY"))

# NOT USING RIGHT NOW
# model_type = "gemini"
llm3 = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.1)

# safety_settings={
#     HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
# }

# # original method
# og_generate = ChatGoogleGenerativeAI._generate

# # patch

# ChatGoogleGenerativeAI._generate = partial(llm._generate, safety_settings=safety_settings)

In [8]:
# Pydantic
class StoryInfo(BaseModel):
    """Get overall information about the story."""
    numChapters: int = Field(description="Number of chapters in the story")
    firstline: str = Field(description="First line of the story")
    lastline: Optional[str] = Field(description="Last line of the story or first volume")
    markers: list[str] = Field(description="List of markers to split the story into chapters, as formatted in the table of contents. Make sure the formatting matches the markers in the actual text (including the word 'chapter' if applicable). Keep all punctuation marks and capitalization, and use the full chapter names. If there are multiple volumes, focus on the first volume.")

def get_marker_prompt(story):
    prompt = f'''List the markers needed to split this story into chapters: {story}. Format your response like Story Info:
    class StoryInfo(BaseModel):
        """Get overall information about the story."""
        firstline: str = Field(description="First line of the story")
        lastline: Optional[str] = Field(description="Last line of the story or first volume")
        markers: list[str] = Field(description="List of markers to split the story into chapters. Keep all punctuation marks and capitalization.")
    '''
    return prompt

# structured_llm = llm2.with_structured_output(StoryInfo)
structured_llm = llm.with_structured_output(StoryInfo)

## split text into chapters

In [2848]:
# NOTE: Make sure your story file is in the scripts folder (as a txt file)
# read in txt file from scripts folder (inside notebooks folder)
og_story_name = "wizard" # name of the story (should match the name of the file (e.g., type "color" for color.txt)
story_name = og_story_name
analysis_type = "character" # character, theme

In [2849]:
# optional, but this part needs to be manually looked up
# for image, provide a url or a file path to the image
story_info = {
"title": "The Wonderful Wizard of Oz",
"type": "Book",
"author": "L. Frank Baum",
"year": 1900,
"url": "https://www.gutenberg.org/ebooks/55",
"image": "https://m.media-amazon.com/images/I/616u3wbjRYL._AC_UF1000,1000_QL80_.jpg",
}

In [2850]:
# read in the script
with open(f"scripts/{story_name}.txt") as f:
    story = f.read()

In [2851]:
if analysis_type == "theme":
    story_name += "_themes"
elif analysis_type == "2":
    story_name += "_2"

In [2852]:
if not os.path.exists(f"json/{story_name}"):
    os.makedirs(f"json/{story_name}")

In [2566]:
# focus on the first volume if there are multiple volumes
if "END OF VOL." in story:
    volumes = story.split("END OF VOL.")
    story = volumes[0]
elif "VOLUME II." in story:
    volumes = story.split("VOLUME II.")
    story = volumes[0]

In [2567]:
# split story into chunks
char_chunk = 120000
story_len = len(story)
print(story_len)
if story_len > char_chunk:
    # story_chunks = [story[i:i + char_chunk] for i in range(0, len(story), char_chunk)]
    # take 0:char_chunk/2 and -char_chunk/2: for the first chunk
    chunk = story[0:char_chunk//2] + story[-char_chunk//2:]
    story_chunks = [chunk]
else: # if story is less than 120k characters, use the whole story
    story_chunks = [story]
print(len(story_chunks))

197254
1


In [2568]:
all_res = []
for i, chunk in enumerate(story_chunks):
    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            # if model_type == 'gemini':
            #     prompt = get_marker_prompt(story)
            #     markers = llm3.invoke(prompt)
            #     res = format_gemini_response(markers)
            #     print(res)
            #     res = DotDict(res['Story Info'])
            # else:
            res = structured_llm.invoke(chunk)
            all_res.append(res)
            print(f"Chunk {i}")
            invalid_input = False
        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            invalid_input = True
    # if i < len(story_chunks) - 1:
    #     # Pause for a specified time (e.g., 60 seconds) to avoid rate limits
    #     time.sleep(60)  # Adjust the sleep duration as per your rate limit requirements

Chunk 0


In [2569]:
# save first line from first chunk, last line from last chunk, and combine markers
first_line = all_res[0].firstline
last_line = all_res[-1].lastline
num_chapters = sum([res.numChapters for res in all_res])
markers = []
for res in all_res:
    markers.extend(res.markers)

num_chapters, markers

(5, ['ACT I', 'ACT II', 'ACT III', 'ACT IV', 'ACT V'])

In [2570]:
# first_line = res.firstline
# last_line = res.lastline
# markers = res.markers
first_line, last_line

('THE TRAGEDY OF HAMLET, PRINCE OF DENMARK',
 '[_Exeunt, bearing off the bodies, after which a peal of ordnance is shot off._]')

In [2853]:
story = re.sub(r' {2,}', ' ', story)  # Replace consecutive spaces with one space
story = re.sub(r'\n{2,}', '\n', story)  # Replace consecutive newlines with one newline
story = re.sub(r' ?\n ?', '\n', story)  # Replace space + newline or newline + space with one newline
story = story.replace("“", "\"").replace("”", "\"").replace("‘", "'").replace("’", "'")

In [2854]:
story_formatted = story.replace("\n", " ")
# replace weird quotes too
story_formatted = story_formatted.replace("\"", " ").replace("'", " ")

In [2573]:
# # remove any marker that doesn't include one of these words
# exclude_words = ["part", "preface", "contents", "by", "end", "epilogue", "letter"]
exclude_words = ["part", "preface", "contents", "introduction", "end", "epilogue", "letter", "volume"]
markers = [m for m in markers if not any(word in m.lower() for word in exclude_words)]

In [2574]:
# save as json 
story_json = {
    "first_line": first_line,
    "last_line": last_line,
    "markers": markers
}

if not os.path.exists(f"json/{og_story_name}"):
    os.makedirs(f"json/{og_story_name}")
with open(f"json/{og_story_name}/summary.json", "w") as f:
    json.dump(story_json, f, indent=4)

In [2855]:
# read in markers from summary.json
with open(f"json/{og_story_name}/summary.json") as f:
    summary = json.load(f)
    markers = summary["markers"]
    first_line = summary["first_line"]
    last_line = summary["last_line"]

In [2584]:
story_formatted[-100:] 

'help produce our new eBooks, and how to subscribe to our email newsletter to hear about new eBooks. '

In [2585]:
first_ind = story_formatted.find(first_line.replace("\"", " ").replace("'", " "))
first_ind, first_line.replace("\"", " ").replace("'", " ")

(2275, 'SCENE I. Elsinore. A platform before the Castle.')

In [2586]:
second_marker = markers[1] + " "
last_line_index = story_formatted.find(second_marker, first_ind)
last_line_index, second_marker, story_formatted[last_line_index-50:last_line_index+50]

(40901,
 'ACT II ',
 ' right. Nay, come, let s go together. [_Exeunt._] ACT II SCENE I. A room in Polonius s house. Enter ')

In [2587]:
last_ind = story_formatted.find(last_line)
last_ind, last_line

(176671,
 '[_Exeunt, bearing off the bodies, after which a peal of ordnance is shot off._]')

In [2588]:
# split story into chapters based on markers
chapters = []
cur_first_line = first_line.replace("\"", " ").replace("'", " ")
cur_min_index = 0
for i, marker in enumerate(markers):
    # print("marker", marker)
    exists_next = i+1 < len(markers)
    next_marker = markers[i+1] if exists_next else ""
    # print("next_marker", next_marker)
    
    if exists_next and " ACT " in next_marker.upper():
        next_marker = next_marker.upper()
    formatted_last_line = next_marker if exists_next else last_line
    # print("formatted_last_line", formatted_last_line)

    if " ACT " not in next_marker.upper():
        formatted_last_line += " "
    cur_last_line = formatted_last_line
    cur_last_line = cur_last_line.replace("\"", " ").replace("'", " ")
    first_line_index = story_formatted.rfind(cur_first_line, cur_min_index)
    print("cur_first_line", repr(cur_first_line))
    print("first_line_index", first_line_index)
    print("Snippet around first_line_index:", repr(story_formatted[first_line_index:first_line_index + 200]))

    if i != 0 and " ACT " not in marker.upper():
        first_line_index += len(cur_first_line)
    last_line_index = story_formatted.find(cur_last_line, first_line_index)
    print("cur_last_line", repr(cur_last_line))
    print("last_line_index", last_line_index)
    print("Snippet around last_line_index:", repr(story_formatted[last_line_index-200:last_line_index]))
    
    if i+1 == len(markers):
        if last_line_index == -1:
            last_line_index = len(story_formatted)
        else:
            last_line_index += len(cur_last_line) 
    chapter = story[first_line_index:last_line_index].strip()

    # add line number in front of each line
    chapter = chapter.split("\n")
    for j, line in enumerate(chapter):
        chapter[j] = f"LINE {j+1}: {line}"
    chapter = "\n".join(chapter)
    chapters.append(chapter)
    cur_first_line = cur_last_line
    cur_min_index = last_line_index

cur_first_line 'SCENE I. Elsinore. A platform before the Castle.'
first_line_index 2275
Snippet around first_line_index: 'SCENE I. Elsinore. A platform before the Castle. Enter Francisco and Barnardo, two sentinels. BARNARDO. Who s there? FRANCISCO. Nay, answer me. Stand and unfold yourself. BARNARDO. Long live the King!'
cur_last_line 'ACT II '
last_line_index 40901
Snippet around last_line_index: 'not lack. Let us go in together, And still your fingers on your lips, I pray. The time is out of joint. O cursed spite, That ever I was born to set it right. Nay, come, let s go together. [_Exeunt._] '
cur_first_line 'ACT II '
first_line_index 40901
Snippet around first_line_index: 'ACT II SCENE I. A room in Polonius s house. Enter Polonius and Reynaldo. POLONIUS. Give him this money and these notes, Reynaldo. REYNALDO. I will, my lord. POLONIUS. You shall do marvellous wisely, g'
cur_last_line 'ACT III '
last_line_index 73569
Snippet around last_line_index: 'd my melancholy, As he is very 

In [2589]:
# check results
for i, marker in enumerate(markers):
    print(f"Chapter {marker}")
    chapter_first_line = chapters[i][:100]
    chapter_last_line = chapters[i][-100:]
    print("first_line:", chapter_first_line)
    print("last_line:", chapter_last_line)
    print("\n")

Chapter ACT I
first_line: LINE 1: SCENE I. Elsinore. A platform before the Castle.
LINE 2: Enter Francisco and Barnardo, two s
last_line: hat ever I was born to set it right.
LINE 1204: Nay, come, let's go together.
LINE 1205: [_Exeunt._]


Chapter ACT II
first_line: LINE 1: SCENE I. A room in Polonius's house.
LINE 2: Enter Polonius and Reynaldo.
LINE 3: POLONIUS.

last_line: s. The play's the thing
LINE 912: Wherein I'll catch the conscience of the King.
LINE 913: [_Exit._]


Chapter ACT III
first_line: LINE 1: SCENE I. A room in the Castle.
LINE 2: Enter King, Queen, Polonius, Ophelia, Rosencrantz and
last_line: rd an end with you.
LINE 1192: Good night, mother.
LINE 1193: [_Exit Hamlet dragging out Polonius._]


Chapter ACT IV
first_line: LINE 1: SCENE I. A room in the Castle.
LINE 2: Enter King, Queen, Rosencrantz and Guildenstern.
LINE
last_line: 8: Now fear I this will give it start again;
LINE 909: Therefore let's follow.
LINE 910: [_Exeunt._]


Chapter ACT V
first_line: LINE 1

In [2590]:
# save to new txt files
# create folder for story if it doesn't exist
if not os.path.exists(f"chapters/{og_story_name}"):
    os.makedirs(f"chapters/{og_story_name}")
for i, marker in enumerate(markers):
    with open(f"chapters/{og_story_name}/{marker}.txt", "w") as f:
        f.write(chapters[i])
        print(f"Chapter {marker} saved.")

Chapter ACT I saved.
Chapter ACT II saved.
Chapter ACT III saved.
Chapter ACT IV saved.
Chapter ACT V saved.


In [2856]:
# read in chapters from txt files
chapters = []
for i, marker in enumerate(markers):
    with open(f"chapters/{og_story_name}/{marker}.txt") as f:
        chapter = f.read()
        chapters.append(chapter)

In [2857]:
chapters[0]

'LINE 1: Dorothy lived in the midst of the great Kansas prairies, with Uncle\nLINE 2: Henry, who was a farmer, and Aunt Em, who was the farmer\'s wife. Their\nLINE 3: house was small, for the lumber to build it had to be carried by wagon\nLINE 4: many miles. There were four walls, a floor and a roof, which made one\nLINE 5: room; and this room contained a rusty looking cookstove, a cupboard for\nLINE 6: the dishes, a table, three or four chairs, and the beds. Uncle Henry\nLINE 7: and Aunt Em had a big bed in one corner, and Dorothy a little bed in\nLINE 8: another corner. There was no garret at all, and no cellar—except a\nLINE 9: small hole dug in the ground, called a cyclone cellar, where the family\nLINE 10: could go in case one of those great whirlwinds arose, mighty enough to\nLINE 11: crush any building in its path. It was reached by a trap door in the\nLINE 12: middle of the floor, from which a ladder led down into the small, dark\nLINE 13: hole.\nLINE 14: When Dorothy stood in 

## analyze scene

In [27]:
scene_type = "character" # location, character, or n/a

In [28]:
extra_scene_instruction = "Analyze the scenes in this chapter."
if scene_type == "location":
    extra_scene_instruction += " Start a new scene when the location changes."
elif scene_type == "character":
    extra_scene_instruction += " Start a new scene when new characters enter or old characters leave."

extra_scene_instruction += "\nChapter text:"

extra_scene_instruction

'Analyze the scenes in this chapter. Start a new scene when new characters enter or old characters leave.\nChapter text:'

In [29]:
# Pydantic
class SceneInfo(BaseModel):
    """Get information about a scene in the story."""
    scene: str = Field(description="Short title of scene (don't include chapter name or number). Each scene should have a unique title, and each title should be no longer than 6 words.")
    summary: str = Field(description="1 line summary of the scene")
    firstline: int = Field(description="First line number of the current scene in this chapter (should be after last line number of previous scene)")
    lastline: int = Field(description="Last line number of the current scene in this chapter (should be after first line number of current scene)")

class SceneListInfo(BaseModel):
    """Get overall information about the story."""
    scenelist: list[SceneInfo] = Field(description="List of key scenes in this chapter")

def get_scenelist_prompt(chapter):
    scene_prompt = f'''List the key scenes in this chapter: {chapter}. Format your response like SceneListInfo:
                    class SceneInfo(BaseModel):
                        """Get information about a scene in the story."""
                        scene: str = Field(description="Title of scene (don't include chapter name or number)")
                        summary: str = Field(description="1 line summary of the scene")
                        firstline: int = Field(description="First line number of the current scene in this chapter (should be after last line number of previous scene)")
                        lastline: int = Field(description="Last line number of the current scene in this chapter (should be after first line number of current scene)")

                    class SceneListInfo(BaseModel):
                        """Get overall information about the story."""
                        scenelist: list[SceneInfo] = Field(description="List of key scenes in this chapter")'''
    return scene_prompt

scene_llm = llm.with_structured_output(SceneListInfo)
# scene_llm = llm2.with_structured_output(SceneListInfo)

In [32]:
# Pydantic
class CharacterInfo(BaseModel):
    """Get information about a character in this scene."""
    name: str = Field(description="Full name of the character. Character name should be 5 words or less.")
    role: str = Field(description="Main actions and/or motivations of the character in this scene")
    importance: float = Field(description="Importance of the character in this scene from from 0: not important at all to 1: very important character")
    emotion: str = Field(description="Emotion of the character in this scene, described in a few words or a short phrase")
    sentiment: float = Field(description="Sentiment of the character in this scene from -1 (very negative) to 1 (very positive)")
    quote: str = Field(description="A direct quote from the character in this scene to illustrate their emotions and/or actions")
    fake_quote: str = Field(description="A brief explanation of the character's emotions and/or actions in this scene if you cannot find a direct quote in the text. If you can find a direct quote, leave this field empty.")

# Pydantic
class ThemeInfo(BaseModel):
    """Get information about a theme in this scene."""
    name: str = Field(description="Short general phrase to describe the theme. Theme name should be 3 words or less.")
    role: str = Field(description="Short description of the role of the theme in this scene")
    importance: float = Field(description="Importance of the theme in this scene from from 0: not important at all to 1: very important theme")
    emotion: str = Field(description="Emotion associated with the theme in this scene, described in a few words or a short phrase")
    sentiment: float = Field(description="Sentiment of the theme in this scene from -1 (very negative) to 1 (very positive)")
    quote: str = Field(description="A direct quote in this scene to illustrate the theme")
    fake_quote: str = Field(description="A brief explanation of the theme in this scene if you cannot find a direct quote in the text. If you can find a direct quote, leave this field empty.")

class SceneDetails(BaseModel):
    """Get overall information about the scene."""
    location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
    importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
    conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
    characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters. Don't include any unnamed characters.")

class SceneDetailsTheme(BaseModel):
    """Get overall information about the scene."""
    location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
    importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
    conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
    themes: list[ThemeInfo] = Field(description="List of key themes in the scene. Make sure there are no duplicates but try to make the themes as general as possible.")

def get_scenedetails_prompt(scene):
    scene_prompt = f'''List the key details in this scene: {scene}. Format your response like SceneDetails:
                    class CharacterInfo(BaseModel):
                        """Get information about a character in this scene."""
                        name: str = Field(description="Full name of the character. Character name should be 5 words or less.")
                        role: str = Field(description="Main actions and/or motivations of the character in this scene")
                        importance: float = Field(description="Importance of the character in this scene from from 0: not important at all to 1: very important character")
                        emotion: str = Field(description="Emotion of the character in this scene, described in a few words or a short phrase")
                        sentiment: float = Field(description="Sentiment of the character in this scene from -1 (very negative) to 1 (very positive)")
                        quote: str = Field(description="A direct quote from the character in this scene to illustrate their emotions and/or actions")

                    class SceneDetails(BaseModel):
                        """Get overall information about the scene."""
                        location: str = Field(description="Physical location of the scene. Location name should be 5 words or less.")
                        importance: float = Field(description="Importance of the scene from 0: not important at all to 1: very important and pivotal scene")
                        conflict: float = Field(description="Conflict in the scene from 0: no conflict to 1: high conflict")
                        characters: list[CharacterInfo] = Field(description="List of key characters in the scene. Make sure there are no duplicates, and focus on speaking characters.  Don't include any unnamed characters.")'''
    return scene_prompt

scene_details_llm = llm.with_structured_output(SceneDetails)
scene_details_llm_theme = llm.with_structured_output(SceneDetailsTheme)
# scene_details_llm = llm2.with_structured_output(SceneDetails)

In [1064]:
async def get_scene_info_async(chapters, chapter_num):
    print(f"Splitting Chapter {markers[chapter_num]} into scenes...")

    cur_chapter = chapters[chapter_num]

    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            if model_type == "gemini":
                prompt = get_scenelist_prompt(cur_chapter)
                chapter_scenes = await llm3.ainvoke(prompt)
                chapter_scenes = format_gemini_response(chapter_scenes)
            else:
                chapter_scenes = await scene_llm.ainvoke(extra_scene_instruction + "\n" + cur_chapter)
            invalid_input = False
        except:
            invalid_input = True
            
    if model_type == "gemini":
        chapter_scenes_list = [DotDict(scene) for scene in chapter_scenes['scenelist']]
    else: 
        chapter_scenes_list = chapter_scenes.scenelist
    formatted_chapter_scenes = []

    cur_last_line = 0

    for scene in chapter_scenes_list:
        scene_title = scene.scene
        scene_summary = scene.summary
        scene_first_line = scene.firstline
        scene_last_line = scene.lastline

        print(f"FIRST LINE: {scene_first_line}")
        print(f"LAST LINE: {scene_last_line}")
        print()

        formatted_scene = {
            "title": scene_title,
            "summary": scene_summary,
            "chapter": markers[chapter_num],
            "first_line": scene_first_line,
            "last_line": scene_last_line
        }

        if scene_last_line > scene_first_line and scene_first_line > cur_last_line:
            # only add scene if last_line > first_line and first_line > cur_last_line
            formatted_chapter_scenes.append(formatted_scene)
            formatted_chapter = cur_chapter.replace("\n", " ")
            # replace weird quotes too
            formatted_chapter = formatted_chapter.replace("\"", " ").replace("'", " ")
            cur_last_line = scene_last_line

    # sort scenes by first_line
    formatted_chapter_scenes = sorted(formatted_chapter_scenes, key=lambda x: x["first_line"])

    # split chapter into scenes based on formatted_chapter_scenes
    scenes = []
    cur_min_index = 0
    for i, scene in enumerate(formatted_chapter_scenes):
        cur_first_line = f"LINE {scene['first_line']}:"
        # cur_first_line = formatted_chapter_scenes[i]["first_line"].strip("...").replace("\"", " ").replace("'", " ")
        first_line_index = formatted_chapter.find(cur_first_line, cur_min_index)
        if first_line_index == -1:
            # see if first_line_index overlaps with last_line of previous scene
            other_possible_first_index = formatted_chapter.find(cur_first_line)
            if other_possible_first_index != -1:
                overlap = cur_chapter[other_possible_first_index:cur_min_index]
                print(f"SCENE {i}")
                print("\nPREVIOUS SCENE")
                print(scenes[i-1])
                print("\nOVERLAP")
                print(overlap)
                print()
            first_line_index = cur_min_index
        cur_last_line = f"LINE {scene['last_line'] + 1}:"
        # cur_last_line = formatted_chapter_scenes[i]["last_line"].strip("...").replace("\"", " ").replace("'", " ")
        last_line_index = formatted_chapter.find(cur_last_line, first_line_index)
        # last_line_index += len(cur_last_line)
        if last_line_index == -1:
            last_line_index = len(cur_chapter)
        scene_text = cur_chapter[first_line_index:last_line_index].strip()
        # remove line numbers
        scene_text = re.sub(r'LINE \d+: ', '', scene_text)
        scenes.append(scene_text)
        cur_min_index = last_line_index

    # add scene text to formatted_chapter_scenes
    for i, scene in enumerate(formatted_chapter_scenes):
        formatted_chapter_scenes[i]["text"] = scenes[i]

    return scenes, formatted_chapter_scenes

In [2791]:
async def get_scene_details_async(scenes, formatted_chapter_scenes, chapter_num):
    print(f"Generating scene details for Chapter {markers[chapter_num]}...")
    all_scene_details = []

    async def process_scene(scene_text, cur_scene):
        """Async function to analyze a single scene"""
        if len(scene_text) == 0:
            return
        
        start_prompt = """
                          When analyzing this scene, focus on characters with a significant role in the scene.
                          If you can't determine the character's role or emotion, don't include them in the analysis.

                          Make sure to only use direct quotes from the text.
                          Don't paraphrase or change the text in any way.
                          If the character speaks, the quote should be something they say.
                          If the character is only described, the quote should be a description of the character's actions or emotions
                          (or any quote that mentions them, even if it is said by another character).
                          Try your best to find a direct quote for each character, but if you can't find a quote, 
                          (1) set the "quote" field to "No quote available" and (2) in the "fake_quote" field,
                          write a brief explanation of the character's emotions and actions in the scene.
                          Leave the "fake_quote" field empty if you can find a direct quote.
                        
                          Scene text:

                          """
        if analysis_type == "theme":
            start_prompt = """
                            When analyzing this scene, focus on themes with a significant role in the scene.
                            If you can't determine the theme's role or emotion, don't include it in the analysis.

                            Make sure to only use direct quotes from the text that describe the theme or relate to it.
                            Don't paraphrase or change the text in any way.
                            Try your best to find a direct quote for each theme, but if you can't find a quote,
                            (1) set the "quote" field to "No quote available" and (2) in the "fake_quote" field,
                            write a brief explanation of the theme in this scene.
                            Leave the "fake_quote" field empty if you can find a direct quote.
                        
                            Scene text:

                            """
            
        invalid_input = True
        while invalid_input:
            try:
                if analysis_type == "theme":
                    scene_details = await scene_details_llm_theme.ainvoke(start_prompt + cur_scene)
                else:
                    scene_details = await scene_details_llm.ainvoke(start_prompt + cur_scene)
                invalid_input = False
                # print(scene_details)
            except:
                invalid_input = True
        
        return scene_details

    # Run all scene details in parallel
    tasks = [process_scene(formatted_chapter_scenes[i]["text"], scenes[i]) for i in range(len(scenes))]
    all_scene_details = await asyncio.gather(*tasks)

    # add details to formatted_chapter_scenes
    for i, scene in enumerate(formatted_chapter_scenes):
        scene_text = formatted_chapter_scenes[i]["text"]
        text_length = len(scene_text)
        num_lines = formatted_chapter_scenes[i]["last_line"] - formatted_chapter_scenes[i]["first_line"] + 1
        # first_line = formatted_chapter_scenes[i]["first_line"]
        # first_line_length = len(first_line)
        # last_line = formatted_chapter_scenes[i]["last_line"]
        # last_line_length = len(last_line)

        # if text_length == 0 or text_length < first_line_length + last_line_length:
        if text_length == 0:
            # no scene text or scene text is too short
            continue
        
        cur_scene_details = all_scene_details[i]
        formatted_chapter_scenes[i]["length"] = text_length
        formatted_chapter_scenes[i]["num_lines"] = num_lines
        formatted_chapter_scenes[i]["location"] = cur_scene_details.location
        formatted_chapter_scenes[i]["importance"] = cur_scene_details.importance
        formatted_chapter_scenes[i]["conflict"] = cur_scene_details.conflict
        formatted_character_info = []

        characters = cur_scene_details.themes if analysis_type == "theme" else cur_scene_details.characters
        # if model_type == "gemini":
        #     characters = [DotDict(character) for character in characters]
        for character in characters:
            character_info = {
                "name": character.name,
                "role": character.role,
                "importance": character.importance,
                "emotion": character.emotion,
                "sentiment": character.sentiment,
                "quote": character.quote,
                "fake_quote": character.fake_quote
            }
            # add character only if role is not N/A and emotion is not N/A and quote is not "No quote available"
            if not (character.role == "N/A" and character.emotion == "N/A" and "No quote available" in character.quote):
                formatted_character_info.append(character_info)
        formatted_chapter_scenes[i]["characters"] = formatted_character_info

    return formatted_chapter_scenes

In [2792]:
async def process_all_chapters_async():
    # Step 1: Split each chapter into scenes in parallel
    chapter_tasks = [get_scene_info_async(chapters, i) for i in range(len(chapters))]
    chapter_results = await asyncio.gather(*chapter_tasks)

    # extract scenes and formatted_chapter_scenes
    scenes = [result[0] for result in chapter_results]
    formatted_chapter_scenes = [result[1] for result in chapter_results]

    # Step 2: Get scene details for each scene in parallel
    detail_tasks = [semaphore_wrapper(get_scene_details_async, scenes[i], formatted_chapter_scenes[i], i) for i in range(len(chapters))]
    final_formatted_scenes = await asyncio.gather(*detail_tasks)

    # Step 3: Filter and save results
    for i, marker in enumerate(markers):
        formatted_chapter_scenes = final_formatted_scenes[i]

        # remove scenes with no text
        formatted_chapter_scenes = [scene for scene in formatted_chapter_scenes if scene["text"] != "" and "characters" in scene]
        # remove characters with no quote
        for scene in formatted_chapter_scenes:
            scene["characters"] = [character for character in scene["characters"] if "(" not in character["quote"] != "" and "<" not in character["name"] and "unnamed" not in character["name"].lower()]
            # print(scene)
        
        # remove scenes with no characters
        formatted_chapter_scenes = [scene for scene in formatted_chapter_scenes if len(scene["characters"]) > 0]

        # save as json
        with open(f"json/{story_name}/{marker}.json", "w") as f:
            json.dump(formatted_chapter_scenes, f, indent=4)

        print(f"All details for Chapter {marker} saved.\n")

In [2858]:
# RUN MAIN SCENE LOOP ASYNC that will get all the scene/character/etc. info for each chapter
await process_all_chapters_async()

Splitting Chapter Chapter I The Cyclone into scenes...
Splitting Chapter Chapter II The Council with the Munchkins into scenes...
Splitting Chapter Chapter III How Dorothy Saved the Scarecrow into scenes...
Splitting Chapter Chapter IV The Road Through the Forest into scenes...
Splitting Chapter Chapter V The Rescue of the Tin Woodman into scenes...
Splitting Chapter Chapter VI The Cowardly Lion into scenes...
Splitting Chapter Chapter VII The Journey to the Great Oz into scenes...
Splitting Chapter Chapter VIII The Deadly Poppy Field into scenes...
Splitting Chapter Chapter IX The Queen of the Field Mice into scenes...
Splitting Chapter Chapter X The Guardian of the Gates into scenes...
Splitting Chapter Chapter XI The Wonderful City of Oz into scenes...
Splitting Chapter Chapter XII The Search for the Wicked Witch into scenes...
Splitting Chapter Chapter XIII The Rescue into scenes...
Splitting Chapter Chapter XIV The Winged Monkeys into scenes...
Splitting Chapter Chapter XV The Dis

In [2859]:
# rank each scene by importance
# and within each scene, the characters

for i, marker in enumerate(markers):
    print(marker)
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    importances = []
    conflicts = []
    
    # extract importance from each scene
    for j, scene in enumerate(scenes):
        importances.append((j, scene["importance"]))
        conflicts.append((j, scene["conflict"]))
        # now extract character importances
        character_importances = []
        for k, character in enumerate(scene["characters"]):
            character_importances.append((k, character["importance"]))
        # sort character importances
        sorted_character_importances = sorted(character_importances, key=lambda x: x[1], reverse=True)
        # add importance_rank to each character
        for k, (l, _) in enumerate(sorted_character_importances):
            scenes[j]["characters"][l]["importance_rank"] = k+1
        # add number of each scene
        scenes[j]["number"] = j+1
    # sort importances
    sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
    # add importance_rank to each scene
    for k, (j, _) in enumerate(sorted_importances):
        scenes[j]["importance_rank"] = k+1
    # sort conflicts
    sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
    # add conflict_rank to each scene
    for k, (j, _) in enumerate(sorted_conflicts):
        scenes[j]["conflict_rank"] = k+1
    
    # print results
    # for j, scene in enumerate(scenes):
    #     print(scenes[j]["title"], scenes[j]["importance_rank"])
    #     print("--------------------------------")
    #     for k, character in enumerate(scene["characters"]):
    #         print(character["name"], character["importance_rank"])
    #     print()

    # save as json
    with open(f"json/{story_name}/{marker}.json", "w") as f:
        json.dump(scenes, f, indent=4)

Chapter I The Cyclone
Chapter II The Council with the Munchkins
Chapter III How Dorothy Saved the Scarecrow
Chapter IV The Road Through the Forest
Chapter V The Rescue of the Tin Woodman
Chapter VI The Cowardly Lion
Chapter VII The Journey to the Great Oz
Chapter VIII The Deadly Poppy Field
Chapter IX The Queen of the Field Mice
Chapter X The Guardian of the Gates
Chapter XI The Wonderful City of Oz
Chapter XII The Search for the Wicked Witch
Chapter XIII The Rescue
Chapter XIV The Winged Monkeys
Chapter XV The Discovery of Oz, the Terrible
Chapter XVI The Magic Art of the Great Humbug
Chapter XVII How the Balloon Was Launched
Chapter XVIII Away to the South
Chapter XX The Dainty China Country
Chapter XXI The Lion Becomes the King of Beasts
Chapter XXII The Country of the Quadlings
Chapter XXIII Glinda The Good Witch Grants Dorothy's Wish
Chapter XXIV Home Again


In [2860]:
# combine all chapters into one json file
all_scenes = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    all_scenes += scenes

with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(all_scenes, f, indent=4)

### grouping duplicate characters / location names

In [2861]:
# collect all unique characters and locations for each chapter
all_characters = {}
all_locations = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    for scene in scenes:
        for character in scene["characters"]:
            char = character["name"]
            if char not in all_characters:
                all_characters[char] = character["role"]
        location = scene["location"]
        if location not in all_locations:
            all_locations.append(location)

In [2862]:
len(all_characters), len(all_locations)

(311, 131)

In [2863]:
all_characters

{'Isolation': 'The scene depicts the isolation of Dorothy and her family in the vast prairies.',
 'Despair': 'The scene illustrates the despair of Aunt Em and Uncle Henry due to their harsh living conditions.',
 'Childhood Innocence': "Dorothy's laughter contrasts with the somberness of her guardians, highlighting her innocence.",
 'Imminent Danger': 'The theme highlights the approaching storm and the urgency of the situation.',
 'Family Protection': "The theme emphasizes the family's instinct to protect each other during the crisis.",
 "Nature's Fury": "The theme illustrates the power of nature and its impact on the characters' lives.",
 'Chaos and Turmoil': 'Represents the uncontrollable forces of nature',
 'Isolation and Displacement': "Highlights Dorothy's sudden separation from her familiar world",
 'Transformation and Change': "Signifies the beginning of Dorothy's journey into the unknown",
 'Fear and Loneliness': "Dorothy's emotional state during the storm",
 'Safety and Protect

In [2864]:
all_locations

['Kansas prairies',
 "Uncle Henry's doorstep",
 "Dorothy's House",
 'Inside the house',
 "Dorothy's new home",
 'Land of the Munchkins',
 'Land of Oz',
 "Witch's domain",
 'Munchkin Land',
 "Witch's home",
 'Munchkinland Forest',
 "Dorothy's home",
 "Munchkin's house",
 'Munchkinland',
 'Yellow Brick Road',
 'Uneven Yellow Brick Road',
 'By the roadside',
 'Cornfield with Scarecrow',
 'Great forest with trees',
 'Cottage in the woods',
 'Forest near yellow brick road',
 "Tin Woodman's cottage",
 "Tin Woodman's Cottage",
 'Thick forest path',
 'Forest path',
 'Empty Basket',
 'Thick Woods',
 'Forest Road Encounter',
 'Forest Clearing',
 'On the journey',
 'Along the road',
 'The Land of Oz',
 'Forest campsite',
 'Emerald City Road',
 'Gulf Edge',
 'Dark Forest',
 'Across the tree',
 'Forest by the river',
 'Under the trees',
 'Beside the river',
 'On the river raft',
 'Middle of the river',
 'Riverbank near shore',
 'Riverbank with Scarecrow',
 'Meadow of Poppies',
 'Poppy Field',
 'Nea

In [2865]:
# # create csv file for characters
# import csv

# with open(f"csv/{story_name}_characters.csv", "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerow(["Character", "Role"])
#     for char, role in all_characters.items():
#         writer.writerow([char, role])

In [2866]:
# # create csv file for locations
# with open(f"csv/{story_name}_locations.csv", "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerow(["Location"])
#     for location in all_locations:
#         writer.writerow([location])

In [2867]:
# Assuming your JSON file is stored at 'all.json'
with open(f"json/{story_name}/all.json") as f:
    json_data = json.load(f)

prompt = """
You are given a list of characters with some duplicates or alternate names.
Your task is to group all the alternate names for each unique character under a single full name, using the descriptions as guidance.
Don't keep characters like "Narrator" unless they are never referred to by another name.
Separate characters that are related to others (e.g., "Bob" and "Ms. Doe (Bob's mom)" and "Bob's sister") into separate entries.

First create a new character list by removing all duplicates and alternate names and only keeping the full name for each character.

Then, output a JSON dictionary where the key is each character in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original character list and should only appear once in the output.

For example:
{{
    "Bob Smith": ["Bob", "Bobby", "Robert"],
    "Jane Doe": ["Jane", "Ms. Doe", "Bob's mother"]
}}
"""

theme_prompt = """
You are given a list of themes with some duplicates or alternate names.
Your task is to group all the alternate names for each unique theme under a single name, using the descriptions as guidance.
Your goal is to have as few themes as possible while still capturing all the different ways the themes are described.

First create a new theme list by removing all duplicates and alternate names and only keeping the best description for each theme.

Then, output a JSON dictionary where the key is each theme in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original theme list and should only appear once in the output.

For example:
{{
    "Memory": ["Remembering the past", "Recollection", "Nostalgia"],
    "Friendship and Loyalty": ["Friendship and Companionship", "Making Friends", "Bonds of Friendship", "Loyalty and Friendship"]
}}
"""

location_prompt = """
You are given a list of locations with some duplicates or alternate names.
Your task is to group all the alternate or similiar names for each unique location under a single location name.

First create a new location list by removing all duplicates and alternate names and only keeping the full name for each location.
If one location is inside another or nearby to it, only keep the larger location (e.g., "Bob's bedroom", "Bob's garden", and "Bob Smith's castle" can be listed under "Bob's house").
Then, output a JSON dictionary where the key is each location in your new list and the value is a list of all possible alternate location names.
All locations should be taken directly from the original location list and should only appear once in the output dict.

For example:
{{
    "Los Angeles": ["LA", "City of Angels", "Cafe in Los Angeles"],
    "Bob's house": ["Bob's bedroom", "Bob Smith's garden", "Bob's castle"]
}}
"""

if analysis_type == "theme":
    prompt = theme_prompt

# if using llm2 (claude)
prompt += "Just output the JSON dictionary as the final result without any additional information."
location_prompt += "Just output the JSON dictionary as the final result without any additional information."

prompt_template = ChatPromptTemplate.from_messages([("system", prompt), ("human", "{input}")])
location_prompt_template = ChatPromptTemplate.from_messages([("system", location_prompt), ("human", "{input}")])

In [2868]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
if analysis_type == "theme":
    final_prompt = prompt_template.invoke(f"theme list: {all_characters}")
else:
    final_prompt = prompt_template.invoke(f"character list: {all_characters}")
# character_llm = llm.with_structured_output(None, method="json_mode") if model_type == "gpt" else llm
character_llm = llm2
# character_llm = llm
output = character_llm.invoke(final_prompt)


In [2869]:
output

AIMessage(content='{\n    "Isolation": ["Isolation", "Isolation and Displacement"],\n    "Despair": ["Despair", "Despair of Homecoming"],\n    "Childhood Innocence": ["Childhood Innocence", "Innocence"],\n    "Imminent Danger": ["Imminent Danger", "Danger", "Danger of the Journey"],\n    "Family Protection": ["Family Protection", "Protection of Loved Ones"],\n    "Nature\'s Fury": ["Nature\'s Fury", "Nature\'s Hostility", "Nature\'s Threat"],\n    "Chaos and Turmoil": ["Chaos and Turmoil"],\n    "Transformation and Change": ["Transformation and Change", "Transformation"],\n    "Fear and Loneliness": ["Fear and Loneliness", "Loneliness", "Loneliness and Isolation"],\n    "Safety and Protection": ["Safety and Protection", "Protection"],\n    "Acceptance of Uncertainty": ["Acceptance of Uncertainty"],\n    "Wonder and Amazement": ["Wonder and Amazement", "Wonder and Curiosity"],\n    "Beauty of Nature": ["Beauty of Nature", "Nature\'s Beauty", "Nature\'s Bounty"],\n    "Escape from Hardsh

In [2870]:
# if model_type == "claude":
output_formatted = output.content
# if output stop_reason = "max_tokens",cut off at last ']' character and add '}'
if output_formatted[-1] != "}":
    last_bracket_ind = output_formatted.rfind("]")
    output_formatted = output_formatted[:last_bracket_ind + 1] + "}"
    # print(output_formatted)
# turn into json
output = json.loads(output_formatted)

In [2871]:
output

{'Isolation': ['Isolation', 'Isolation and Displacement'],
 'Despair': ['Despair', 'Despair of Homecoming'],
 'Childhood Innocence': ['Childhood Innocence', 'Innocence'],
 'Imminent Danger': ['Imminent Danger', 'Danger', 'Danger of the Journey'],
 'Family Protection': ['Family Protection', 'Protection of Loved Ones'],
 "Nature's Fury": ["Nature's Fury", "Nature's Hostility", "Nature's Threat"],
 'Chaos and Turmoil': ['Chaos and Turmoil'],
 'Transformation and Change': ['Transformation and Change', 'Transformation'],
 'Fear and Loneliness': ['Fear and Loneliness',
  'Loneliness',
  'Loneliness and Isolation'],
 'Safety and Protection': ['Safety and Protection', 'Protection'],
 'Acceptance of Uncertainty': ['Acceptance of Uncertainty'],
 'Wonder and Amazement': ['Wonder and Amazement', 'Wonder and Curiosity'],
 'Beauty of Nature': ['Beauty of Nature',
  "Nature's Beauty",
  "Nature's Bounty"],
 'Escape from Hardship': ['Escape from Hardship'],
 'Welcome and Gratitude': ['Welcome and Grat

In [2872]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
final_location_prompt = location_prompt_template.invoke(f"location list: {all_locations}")
# location_llm = llm.with_structured_output(None, method="json_mode") if model_type == "gpt" else llm
location_llm = llm2
# location_llm = llm
location_output = location_llm.invoke(final_location_prompt)

In [2873]:
location_output

AIMessage(content='{\n    "Kansas": ["Kansas prairies", "Kansas prairie", "Aunt Em\'s house", "Uncle Henry\'s doorstep"],\n    "Dorothy\'s House": ["Dorothy\'s new home", "Inside the house", "Dorothy\'s home"],\n    "Land of Oz": ["The Land of Oz"],\n    "Munchkinland": ["Land of the Munchkins", "Munchkin Land", "Munchkinland Forest", "Munchkin\'s house"],\n    "Witch\'s domain": ["Witch\'s home", "Witch\'s lair", "Witch\'s cupboard", "Wicked Witch\'s Lair", "Witch\'s castle yard", "Witch\'s Castle", "Witch\'s Lair", "Witch\'s Castle Kitchen"],\n    "Yellow Brick Road": ["Uneven Yellow Brick Road", "By the roadside", "On the yellow brick road", "Beside the road", "Near the yellow brick road"],\n    "Cornfield": ["Cornfield with Scarecrow"],\n    "Forest": ["Great forest with trees", "Cottage in the woods", "Forest near yellow brick road", "Thick forest path", "Forest path", "Thick Woods", "Forest Road Encounter", "Forest Clearing", "Dark Forest", "Forest by the river", "Under the trees

In [2874]:
location_output_formatted = location_output.content
# turn into json
location_output = json.loads(location_output_formatted)

In [2875]:
location_output

{'Kansas': ['Kansas prairies',
  'Kansas prairie',
  "Aunt Em's house",
  "Uncle Henry's doorstep"],
 "Dorothy's House": ["Dorothy's new home",
  'Inside the house',
  "Dorothy's home"],
 'Land of Oz': ['The Land of Oz'],
 'Munchkinland': ['Land of the Munchkins',
  'Munchkin Land',
  'Munchkinland Forest',
  "Munchkin's house"],
 "Witch's domain": ["Witch's home",
  "Witch's lair",
  "Witch's cupboard",
  "Wicked Witch's Lair",
  "Witch's castle yard",
  "Witch's Castle",
  "Witch's Lair",
  "Witch's Castle Kitchen"],
 'Yellow Brick Road': ['Uneven Yellow Brick Road',
  'By the roadside',
  'On the yellow brick road',
  'Beside the road',
  'Near the yellow brick road'],
 'Cornfield': ['Cornfield with Scarecrow'],
 'Forest': ['Great forest with trees',
  'Cottage in the woods',
  'Forest near yellow brick road',
  'Thick forest path',
  'Forest path',
  'Thick Woods',
  'Forest Road Encounter',
  'Forest Clearing',
  'Dark Forest',
  'Forest by the river',
  'Under the trees',
  'Thic

In [2876]:
# Function to find the correct key based on a search string, ensuring whole-word matches
def find_key_from_alias(text, character_dict):
    for key, aliases in character_dict.items():
        for alias in sorted(aliases, key=len, reverse=True):
            # Match the alias as a whole word, and ensure no partial matches within other words
            pattern = rf'\b{re.escape(alias)}\b(?!\'s)'  # (?!\'s) ensures no match for possessives like Mitsuha's
            if re.search(pattern, text):
                return key  # Return the key if the alias is found as a whole word
    return text  # Return the original text if no match is found

In [2877]:
# Load the scenes
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

# Capitalize output and location output
capitalized_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in output.items()}
capitalized_location_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in location_output.items()}

# Process each scene
for scene in scenes:
    # Process characters
    for character in scene["characters"]:
        char = string.capwords(character["name"])
        found_char = False

        # Check if the character's name is a key in the dictionary
        if char in capitalized_output:
            found_char = True
            character["name"] = char  # Capitalize the character name
        
        # If not found in the dictionary, check the aliases
        if not found_char:
            # Find the right key for the alias (if exists)
            matched_key = find_key_from_alias(char, capitalized_output)
            if matched_key != char:
                character["name"] = matched_key
                found_char = True
        if not found_char:
            character["name"] = char

    # Process location
    loc = string.capwords(scene["location"])
    found_loc = False

    # Check if the location is a key in the location dictionary
    if loc in capitalized_location_output:
        found_loc = True
        scene["location"] = loc  # Capitalize the location

    # If not found in the dictionary, check the aliases
    if not found_loc:
        matched_key = find_key_from_alias(loc, capitalized_location_output)
        if matched_key != loc:
            scene["location"] = matched_key
            found_loc = True
    if not found_loc:
        scene["location"] = loc

# Save the updated scenes
with open(f"json/{story_name}/all.json", "w") as f:
    json.dump(scenes, f, indent=4)

In [2878]:
# now list all characters and locations again
# collect all unique characters and locations for each chapter
all_characters_new = {}
all_locations_new = []
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)
for scene in scenes:
    for character in scene["characters"]:
        char = character["name"]
        if char not in all_characters_new:
            all_characters_new[char] = 1
        else:
            all_characters_new[char] += 1
    location = scene["location"]
    if location not in all_locations_new:
        all_locations_new.append(location)

In [2879]:
len(all_characters_new), len(all_locations_new)

(239, 52)

In [2880]:
all_characters_new 

{'Isolation': 8,
 'Despair': 3,
 'Childhood Innocence': 2,
 'Imminent Danger': 11,
 'Family Protection': 2,
 "Nature's Fury": 4,
 'Chaos And Turmoil': 1,
 'Transformation And Change': 8,
 'Fear And Loneliness': 2,
 'Safety And Protection': 6,
 'Acceptance Of Uncertainty': 1,
 'Wonder And Amazement': 2,
 'Beauty Of Nature': 7,
 'Escape From Hardship': 1,
 'Welcome And Gratitude': 10,
 'Curiosity And Wonder': 7,
 'Oddity And Uniqueness': 1,
 'Good Vs Evil': 2,
 'Freedom': 3,
 'Loss And Legacy': 4,
 'Power Of The Shoes': 1,
 'Danger Of The Journey': 1,
 'Journey Ahead': 12,
 'Friendship And Support': 46,
 'Magic And Protection': 6,
 'Fear And Courage': 15,
 'Preparation For Journey': 1,
 'Nostalgia And Change': 1,
 'Independence': 1,
 'Celebration Of Freedom': 1,
 'Misunderstanding Identity': 1,
 'Hospitality': 2,
 'Identity': 4,
 'Adventure': 5,
 'Fear Of Fire': 1,
 'Courage': 9,
 'Struggle': 1,
 'Desolation': 1,
 'Home Vs. Beauty': 1,
 'Desire For Knowledge': 2,
 "Nature's Embrace": 1,


In [2881]:

all_locations_new

['Kansas',
 "Dorothy's House",
 'Munchkinland',
 'Land Of Oz',
 "Witch's Domain",
 'Yellow Brick Road',
 'Cornfield',
 'Forest',
 "Tin Woodman's Cottage",
 'Empty Basket',
 'On The Journey',
 'Along The Road',
 'Forest Campsite',
 'Emerald City Road',
 'Gulf Edge',
 'Across The Tree',
 'River',
 'Poppy Field',
 'Open Field',
 "Queen's Domain",
 'Mice Gathering Area',
 'Poppy Bed',
 'Farmhouse Near Emerald City',
 'Emerald City',
 'Palace Of Oz',
 'Land Of The Winkies',
 'Fields',
 "Wicked Witch's Castle",
 'Rocky Plain',
 "Dorothy's Room",
 'Tall Tree Clearing',
 'Golden Cap',
 'On The Ride',
 "Wizard's Balloon Departure",
 "Dorothy's Emotional Farewell",
 "Tin Woodman's Lament",
 'The City Gate',
 'Land Of The South',
 'Under The Tree',
 'Top Of The Wall',
 'Country Of The China People',
 'Disagreeable Country',
 "Spider's Lair",
 'Steep Hill With Rocks',
 'Country Of The Quadlings',
 "Farmer's House",
 'Beautiful Castle Gates',
 'Castle Room',
 'Throne Room Of Glinda',
 "Dorothy's En

## produce chapter summaries

In [9]:
# Pydantic
class ChapterSummary(BaseModel):
    """Summarize a chapter based on scenes."""
    summary: str = Field(description="A brief, 1-line summary of the chapter")

chapter_llm = llm.with_structured_output(ChapterSummary)

In [ ]:
# remove any markers without any scenes
markers_to_remove = []
for i, marker in enumerate(markers):
    with open(f"json/{story_name}/{marker}.json") as f:
        scenes = json.load(f)
    if len(scenes) == 0:
        markers_to_remove.append(marker)

for marker in markers_to_remove:
    markers.remove(marker)

In [12]:
async def read_json_async(file_path):
    """Asynchronously read a JSON file."""
    async with aiofiles.open(file_path, mode="r") as f:
        data = await f.read()
        return json.loads(data)
    
async def read_text_async(file_path):
    """Asynchronously read a text file."""
    async with aiofiles.open(file_path, mode="r") as f:
        return await f.read()
    
async def process_chapter_summary_async(story_name, og_story_name, marker):
    """Process the summary for a single chapter."""
    scenes_file = f"json/{story_name}/{marker}.json"
    chapter_file = f"chapters/{og_story_name}/{marker}.txt"

    # Read JSON and text files asynchronously
    scenes, chapter_text = await asyncio.gather(read_json_async(scenes_file), read_text_async(chapter_file))

    # Create scene summaries string
    scene_summaries = " ".join([scene["summary"] for scene in scenes])

    # Get summary from LLM
    summary = await chapter_llm.ainvoke(scene_summaries)

    chapter_summary = {
        "chapter": marker,
        "scenes": len(scenes),
        "length": len(chapter_text),
        "num_lines": len(chapter_text.split("\n")),
        "summary": summary.summary,
        "importance": [],
        "conflict": [],
        "locations": {},
        "characters": {}
    }

    return chapter_summary

async def generate_chapter_summaries_async(story_name, og_story_name, markers):
    """Generate summaries for all chapters in parallel."""
    # Processs chapters in parallel
    chapter_summaries = await asyncio.gather(*[process_chapter_summary_async(story_name, og_story_name, marker) for marker in markers])
    print("chapter_summaries", chapter_summaries)
    # Read all.json file asynchronously
    scenes = await read_json_async(f"json/{story_name}/all.json")

    # Aggregate character and location information
    for chapter_summary in chapter_summaries:
        chapter = chapter_summary["chapter"]

        chapter_scenes = [scene for scene in scenes if scene["chapter"] == chapter]
    
        for scene in chapter_scenes:
            # add location to chapter summary
            location = scene["location"]
            chapter_summary_locations = chapter_summary["locations"]
            if location not in chapter_summary_locations:
                chapter_summary_locations[location] = 1
            else:
                chapter_summary_locations[location] += 1
            # add importance to chapter summary
            importance = scene["importance"]
            chapter_summary["importance"].append(importance)
            # add conflict to chapter summary
            conflict = scene["conflict"]
            chapter_summary["conflict"].append(conflict)
            # add characters to chapter summary
            characters = scene["characters"]
            for character in characters:
                char = character["name"]
                chapter_summary_characters = chapter_summary["characters"]
                if char not in chapter_summary_characters:
                    chapter_summary_characters[char] = 1
                else:
                    chapter_summary_characters[char] += 1

    # now average importance and conflict for each chapter
    for chapter_summary in chapter_summaries:
        if len(chapter_summary["importance"]) == 0:
            print(f"Chapter {chapter_summary['chapter']} has no scenes.")
            continue
        chapter_summary["importance"] = round(sum(chapter_summary["importance"]) / len(chapter_summary["importance"]), 2)
        chapter_summary["conflict"] = round(sum(chapter_summary["conflict"]) / len(chapter_summary["conflict"]), 2)

    print("Importances", [chapter_summary["importance"] for chapter_summary in chapter_summaries])
    print("Conflicts", [chapter_summary["conflict"] for chapter_summary in chapter_summaries])

    # rank each chapter by importance and conflict
    importances = []
    conflicts = []
    for chapter_summary in chapter_summaries:
        importances.append((chapter_summary["chapter"], chapter_summary["importance"]))
        conflicts.append((chapter_summary["chapter"], chapter_summary["conflict"]))
        
    # sort importances
    sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
    # add importance_rank to each chaptera
    for k, (j, _) in enumerate(sorted_importances):
        chapter_summaries[markers.index(j)]["importance_rank"] = k+1

    # sort conflicts
    sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
    # add conflict_rank to each chapter
    for k, (j, _) in enumerate(sorted_conflicts):
        chapter_summaries[markers.index(j)]["conflict_rank"] = k+1

    return chapter_summaries

In [ ]:
## FIXING CHAPTER DATA 
# import json 

# async def fix_chapter_data(json_name, story_name):
#     og_story_name = story_name.split("_")[0]
#     # read in markers
#     with open(f"json/{og_story_name}/summary.json") as f:
#         summary = json.load(f)
    
#     markers = summary["markers"]
#     chapter_summaries = await generate_chapter_summaries_async(story_name, og_story_name, markers)

#     # load in final data
#     with open(f"../src/data/{json_name}") as f:
#         final_data = json.load(f)
    
#     # replace chapter data
#     final_data["chapters"] = chapter_summaries

#     # save final data
#     with open(f"../src/data/{json_name}", "w") as f:
#         json.dump(final_data, f, indent=4)
    

# # get all stories
# path = "../src/data/"

# exclude = ["yourname"]

# # get all json files containing "-new"
# json_files = [f for f in os.listdir(path) if ("-new") in f and f.endswith(".json") and not any(e in f for e in exclude)]
# json_files = sorted(json_files)
# stories = [f.replace("-new", "").replace(".json","").replace("-","_") for f in json_files]

# done = ["alice", "anne", "bookstore", "color", "emma", "frankenstein", "gatsby", "greatexp", "hamlet"]

# for json_file, story in zip(json_files, stories):
#     # if story includes any part of a story in done, skip
#     if any(d in story for d in done):
#         continue
#     print(f"Processing {story} and {json_file}...")

#     await fix_chapter_data(json_file, story)

In [2926]:
# create summary json of each chapter
chapter_summaries = await generate_chapter_summaries_async(story_name, og_story_name, markers)

In [2928]:
chapter_summaries

[{'chapter': 'CHAPTER I.',
  'scenes': 9,
  'length': 21537,
  'num_lines': 306,
  'summary': 'Alice follows the White Rabbit down a hole, explores a hall with locked doors, and discovers a key and a bottle that shrinks her, but she forgets the key to the door.',
  'importance': 0.8,
  'conflict': 0.37,
  'locations': {'Riverbank': 1,
   'Rabbit-hole': 1,
   'Falling Through The Air': 1,
   'Hall': 2,
   "Alice's Table": 2,
   'Wonderland': 1,
   "Alice's Garden": 1},
  'characters': {'Alice': 10, 'White Rabbit': 2, 'Dinah': 1},
  'importance_rank': 5,
  'conflict_rank': 10},
 {'chapter': 'CHAPTER II.',
  'scenes': 11,
  'length': 21537,
  'num_lines': 306,
  'summary': 'Alice navigates her feelings of identity and size while swimming in her own tears, leading to an encounter with a Mouse and a group of animals.',
  'importance': 0.71,
  'conflict': 0.43,
  'locations': {'Pool Of Tears': 7, 'Hall': 2, 'Forest': 1, 'Shore': 1},
  'characters': {'Alice': 11,
   'White Rabbit': 1,
   'Mab

In [2887]:
# read in scenes from all_json
with open(f"json/{story_name}/all.json") as f:
    scenes = json.load(f)

In [2888]:
all_json = {}
all_json["title"] = story_info["title"]
all_json["type"] = story_info["type"]
if (all_json["type"] == "Movie"):
    all_json["director"] = story_info["director"]
else:
    all_json["author"] = story_info["author"]
all_json["year"] = story_info["year"]
if "url" in story_info:
    all_json["url"] = story_info["url"]
if "image" in story_info:
    all_json["image"] = story_info["image"]
all_json["num_chapters"] = len(chapters)
all_json["num_scenes"] = len(scenes)
all_json["num_characters"] = len(all_characters_new)
all_json["num_locations"] = len(all_locations_new)
all_json["chapters"] = chapter_summaries
all_json["scenes"] = scenes

## produce character summaries

In [2889]:
# Pydantic
class CharacterInfo(BaseModel):
    """Get information about a character in the story."""
    name: str = Field(description="Name of character.")
    quote: str = Field(description="Direct quote from the story that the character says or that describes them.")
    group: str = Field(description="Group that this character belongs to (e.g., main characters, upper class, talking animals). There should be at least 2 different groups of characters.")
    # color: str = Field(description="Unique aesthetic RGB color string that represents this character (e.g., rgb(118, 185, 71)). Every character should have a different color. Don't use white and make sure the color is visible against a white background.")
    # explanation: str = Field(description="Explanation why this color was chosen for this character.")

class LegendThemeInfo(BaseModel):
    """Get information about a theme in the story."""
    name: str = Field(description="Name of theme.")
    quote: str = Field(description="Direct quote from the story that describes this theme.")
    group: Optional[str] = Field(description="Group that this theme belongs to (e.g., Family, Social Themes, Emotions). There should be at least 2 different groups of themes. Make sure all groups are distinct and don't overlap with each other.")

class CharacterList(BaseModel):
    """Get overall information about the story."""
    # characterlist: list[CharacterInfo] = Field(description="List of character details. Make sure there is exactly one entry per character in the provided list and no additional characters are added. Choose a different color for each character.")
    characterlist: list[CharacterInfo] = Field(description="List of character details. Make sure there is exactly one entry per character in the provided list and no additional characters are added.")

class LegendThemeList(BaseModel):
     """Get overall information about the story."""
     themelist: list[LegendThemeInfo] = Field(description="List of theme details. Make sure there is exactly one entry per theme in the provided list and no additional themes are added.")

class ColorInfo(BaseModel):
    """Pick a color + explain why for each character"""
    name: str = Field(description="Name of character.")
    color: str = Field(description="Unique aesthetic RGB color string that represents this character (e.g., rgb(118, 185, 71)). Every character should have a different color. Don't use white and make sure the color is visible against a white background.")
    explanation: str = Field(description="Explanation why this color was chosen for this character.")

class ThemeColorInfo(BaseModel):
    """Pick a color + explain why for each theme"""
    name: str = Field(description="Name of theme.")
    color: str = Field(description="Unique aesthetic RGB color string that represents this theme (e.g., rgb(118, 185, 71)). Every theme should have a different color. Don't use white and make sure the color is visible against a white background.")
    explanation: str = Field(description="Explanation why this color was chosen for this theme.")

class ColorList(BaseModel):
    """List of color info for characters"""
    colorlist: list[ColorInfo] = Field(description="List of color details for each character. Make sure there is exactly one entry per character in the provided list and no additional characters are added. Choose a different color for each character.")

class ThemeColorList(BaseModel):
    """List of color info for themes"""
    colorlist: list[ThemeColorInfo] = Field(description="List of color details for each theme. Make sure there is exactly one entry per theme in the provided list and no additional themes are added. Choose a different color for each theme.")

if analysis_type == "theme":
    characterlist_llm = llm.with_structured_output(LegendThemeList)
    colorlist_llm = llm.with_structured_output(ThemeColorList)
else:
    characterlist_llm = llm.with_structured_output(CharacterList)
    colorlist_llm = llm.with_structured_output(ColorList)

In [2890]:
# sort all_characters_new by count, descending
sorted_characters = dict(sorted(all_characters_new.items(), key=lambda item: item[1], reverse=True))
# take top 20 characters
top_characters = dict(list(sorted_characters.items())[:20])
top_characters

{'Friendship And Support': 46,
 'Fear And Courage': 15,
 'Journey Ahead': 12,
 'Teamwork And Problem Solving': 12,
 'Imminent Danger': 11,
 'Welcome And Gratitude': 10,
 'Courage': 9,
 'Isolation': 8,
 'Transformation And Change': 8,
 'Beauty Of Nature': 7,
 'Curiosity And Wonder': 7,
 'Safety And Protection': 6,
 'Magic And Protection': 6,
 'Adventure': 5,
 'Leadership': 5,
 'Desire For Home': 5,
 "Nature's Fury": 4,
 'Loss And Legacy': 4,
 'Identity': 4,
 'Resourcefulness': 4}

In [2891]:
character_arr = []
cur_arr = []
max_chars = 20

char_names = list(all_characters_new.keys())
# char_names = list(top_characters.keys())
for i, c in enumerate(char_names):
    cur_arr.append(c)
    if len(cur_arr) == max_chars or i == len(char_names) - 1:
        character_arr.append(cur_arr)
        cur_arr = []

for arr in character_arr:
    print(len(arr))


20
20
20
20
20
20
20
20
20
20
20
19


In [2892]:
async def process_character_chunk(i, arr):
    """Process a single chunk of character assignment asynchronously."""
    invalid_input = True
    while invalid_input:
        try:
            # filter scenes to include only those with relevant characters
            new_scenes = [scene for scene in scenes if any(char["name"] in arr for char in scene["characters"])]

             # take middle 126000 characters of new_scenes
            new_scenes_str = json.dumps(new_scenes)
            new_scenes_str = new_scenes_str[:126000]

            # construct prompt
            prompt = f"""characters: {arr}
                         story info: {new_scenes_str}

                         your output should contain exactly the same {len(arr)} characters as in the original list.

                         remember to only use direct quotes from the text and not paraphrase or change the text in any way.
                         """

            # invoke model
            res = await characterlist_llm.ainvoke(prompt)

            print(f"Chunk {i} processed.")
            return res

        except:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            await asyncio.sleep(1) # wait for 1 second before trying again

async def process_all_characters():
    """Process all characters in parallel."""
    tasks = [process_character_chunk(i, arr) for i, arr in enumerate(character_arr)]
    results = await asyncio.gather(*tasks)
    return results

In [2893]:
# Run the main function to process all characters
all_res = await process_all_characters()

Chunk 11 processed.
Chunk 10 processed.
Chunk 6 processed.
Chunk 8 processed.
Chunk 3 processed.
Chunk 4 processed.
Chunk 9 processed.
Chunk 1 processed.
Chunk 2 processed.
Chunk 5 processed.
Chunk 0 processed.
Chunk 7 processed.


In [2894]:
# combine characterlists into one
characters = []
for res in all_res:
    if analysis_type == "theme":
        print(len(res.themelist))
        characters += res.themelist
    else:
        print(len(res.characterlist))
        characters += res.characterlist

len(characters)

20
20
20
20
20
20
20
20
20
20
20
19


239

In [2895]:
character_names = [char.name for char in characters]
for name in character_names:
    if name not in char_names:
        print("not in orig list:", name)

need_to_add = []
for name in char_names:
    if name not in character_names:
        print("not in llm output:", name)
        need_to_add.append(name)

# find duplicate names
unique_names = []
duplicate_indices = []
for i, name in enumerate(character_names):
    if name not in unique_names:
        unique_names.append(name)
    else:
        print("duplicate:", name, i)
        duplicate_indices.append(i)

print("need_to_add", len(need_to_add))
print("duplicate_indices", len(duplicate_indices))


need_to_add 0
duplicate_indices 0


### group duplicate character groups

In [2896]:
# group similar groups

group_prompt = """
You are given a list of groups with some duplicates or alternate names.
Your task is to group all the alternate names for each unique group under a single name.
Your goal is to have as few groups as possible while still capturing all the unique groups.

First create a new group list by removing all duplicates and alternate names and only keeping the best description for each group.

Then, output a JSON dictionary where the key is each group in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original group list and should only appear once in the output.

For example:
{{
    "Main Characters": ["Main Characters", "Protagonists", "Heroes"],
    "Elders": ["Elders", "Family Authorities"],
    "Exploration": ["Exploration", "Adventure", "Discovery"],
    "Relationships": ["Relationships", "Friendships"]
}}
"""

# if using llm2 (claude)
group_prompt += "Just output the JSON dictionary as the final result without any additional information."

group_prompt_template = ChatPromptTemplate.from_messages([("system", group_prompt), ("human", "{input}")])

In [2897]:
# get all unique groups
# groups = [c['group'] for c in characters]
groups = [c.group for c in characters]
unique_groups = list(set(groups))
len(unique_groups), unique_groups

(46,
 ['Safety',
  'Ambition',
  'Desire',
  'Trust',
  'Support',
  'Strength',
  'Nature',
  'Hope',
  'Quest',
  'Adventure',
  'Determination',
  'Resourcefulness',
  'Struggle',
  'Independence',
  'Change',
  'Fulfillment',
  'Survival',
  'Knowledge',
  'Identity',
  'Magic',
  'Fear',
  'Conflict',
  'Injustice',
  'Moral Themes',
  'Preparation',
  'Friendship',
  'Perception',
  'Social Themes',
  'Emotions',
  'Consequences',
  'Power Dynamics',
  'Manipulation',
  'Leadership',
  'Triumph',
  'Journey',
  'Courage',
  'Comfort',
  'Fantasy',
  'Teamwork',
  'Visual Themes',
  'Fortune',
  'Community',
  'Challenges',
  'Goals',
  'Compassion',
  'Restoration'])

In [2898]:
# Create the LLM chain with the prompt template and model

# Run the chain by passing the characters and JSON data as inputs
if analysis_type == "theme":
    final_prompt = group_prompt_template.invoke(f"list of groups for themes: {unique_groups}")
else:
    final_prompt = group_prompt_template.invoke(f"list of groups for characters: {unique_groups}")
group_llm = llm2
output = group_llm.invoke(final_prompt)

In [2899]:
output

AIMessage(content='{\n  "Safety": ["Safety"],\n  "Ambition": ["Ambition"],\n  "Desire": ["Desire"],\n  "Trust": ["Trust"],\n  "Support": ["Support"],\n  "Strength": ["Strength"],\n  "Nature": ["Nature"],\n  "Hope": ["Hope"],\n  "Adventure": ["Quest", "Adventure", "Journey"],\n  "Determination": ["Determination"],\n  "Resourcefulness": ["Resourcefulness"],\n  "Struggle": ["Struggle", "Challenges"],\n  "Independence": ["Independence"],\n  "Change": ["Change"],\n  "Fulfillment": ["Fulfillment"],\n  "Survival": ["Survival"],\n  "Knowledge": ["Knowledge"],\n  "Identity": ["Identity"],\n  "Magic": ["Magic", "Fantasy"],\n  "Fear": ["Fear"],\n  "Conflict": ["Conflict"],\n  "Injustice": ["Injustice"],\n  "Moral Themes": ["Moral Themes"],\n  "Preparation": ["Preparation"],\n  "Friendship": ["Friendship", "Teamwork"],\n  "Perception": ["Perception"],\n  "Social Themes": ["Social Themes", "Community"],\n  "Emotions": ["Emotions"],\n  "Consequences": ["Consequences"],\n  "Power Dynamics": ["Power D

In [2900]:
output_formatted = output.content
# turn into json
output = json.loads(output_formatted)
len(output), output

(40,
 {'Safety': ['Safety'],
  'Ambition': ['Ambition'],
  'Desire': ['Desire'],
  'Trust': ['Trust'],
  'Support': ['Support'],
  'Strength': ['Strength'],
  'Nature': ['Nature'],
  'Hope': ['Hope'],
  'Adventure': ['Quest', 'Adventure', 'Journey'],
  'Determination': ['Determination'],
  'Resourcefulness': ['Resourcefulness'],
  'Struggle': ['Struggle', 'Challenges'],
  'Independence': ['Independence'],
  'Change': ['Change'],
  'Fulfillment': ['Fulfillment'],
  'Survival': ['Survival'],
  'Knowledge': ['Knowledge'],
  'Identity': ['Identity'],
  'Magic': ['Magic', 'Fantasy'],
  'Fear': ['Fear'],
  'Conflict': ['Conflict'],
  'Injustice': ['Injustice'],
  'Moral Themes': ['Moral Themes'],
  'Preparation': ['Preparation'],
  'Friendship': ['Friendship', 'Teamwork'],
  'Perception': ['Perception'],
  'Social Themes': ['Social Themes', 'Community'],
  'Emotions': ['Emotions'],
  'Consequences': ['Consequences'],
  'Power Dynamics': ['Power Dynamics'],
  'Manipulation': ['Manipulation'],

In [2901]:
# Capitalize output and location output
capitalized_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in output.items()}

# Process each scene
for c in characters:

    # Process group
    group = string.capwords(c.group)
    found_group = False

    # Check if the group is a key in the dictionary
    if group in capitalized_output:
        found_group = True
        c.group = group  # Capitalize the group

    # If not found in the dictionary, check the aliases
    if not found_group:
        matched_key = find_key_from_alias(group, capitalized_output)
        if matched_key != group:
            c.group = matched_key
            found_group = True
    if not found_group:
        c.group = group

### assign colors to characters

In [2902]:
# now add colors for top characters
top_char_names = list(top_characters.keys())

all_res_colors = []
invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    # filter all_json to only include characters in arr
    new_scenes = []
    for scene in scenes:
        include = False
        for character in scene["characters"]:
            if character["name"] in top_char_names:
                include = True
                break
        if include:
            new_scenes.append(scene)

    # take middle 126000 characters of new_scenes
    new_scenes_str = json.dumps(new_scenes)
    new_scenes_str = new_scenes_str[:126000]

    try:
        prompt = f"characters: {top_char_names}\nstory info: {new_scenes_str}\nyour output should contain exactly the same {len(top_char_names)} characters as in the original list. Make sure each character is assigned a unique color."
        res = colorlist_llm.invoke(prompt)
        all_res_colors.append(res)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [2903]:
# find duplicate colors
colorlist = []
for res in all_res_colors:
    colorlist += res.colorlist
    
unique_colors = []
duplicate_colors = []
for i, char in enumerate(colorlist):
    color = char.color
    if color not in unique_colors:
        unique_colors.append(color)
    else:
        print("duplicate color:", color, i)
        duplicate_colors.append(i)

In [2904]:
colorinfo = {}

for c in colorlist:
    colorinfo[c.name] = {
        "color": c.color,
        "explanation": c.explanation
    }

colorinfo

{'Friendship And Support': {'color': 'rgb(255, 182, 193)',
  'explanation': 'This soft pink color represents warmth and compassion, reflecting the supportive relationships among characters.'},
 'Fear And Courage': {'color': 'rgb(255, 99, 71)',
  'explanation': 'The bold red symbolizes fear and urgency, capturing the tension and bravery displayed in dangerous situations.'},
 'Journey Ahead': {'color': 'rgb(135, 206, 250)',
  'explanation': 'This light blue represents hope and the vastness of the journey, evoking a sense of adventure and exploration.'},
 'Teamwork And Problem Solving': {'color': 'rgb(255, 215, 0)',
  'explanation': "The bright yellow signifies optimism and collaboration, highlighting the characters' ability to work together."},
 'Imminent Danger': {'color': 'rgb(128, 0, 128)',
  'explanation': 'The deep purple conveys a sense of foreboding and danger, reflecting the threats the characters face.'},
 'Welcome And Gratitude': {'color': 'rgb(144, 238, 144)',
  'explanation':

In [2905]:
# remove duplicates
characters = [char for i, char in enumerate(characters) if i not in duplicate_indices]
print(len(characters))

239


In [2906]:
character_info = []
for c in characters:

    # see if character is in color_info
    name = c.name
    color = ""
    explanation = ""
    if name in colorinfo:
        color = colorinfo[name]["color"]
        explanation = colorinfo[name]["explanation"]
    
    character_info.append({
        "name": name,
        "quote": c.quote,
        "group": c.group,
        "color": color,
        "explanation": explanation
    })

len(character_info), character_info

(239,
 [{'name': 'Isolation',
   'quote': 'When Dorothy stood in the doorway and looked around, she could see nothing but the great gray prairie on every side.',
   'group': 'Emotions',
   'color': 'rgb(169, 169, 169)',
   'explanation': 'The gray color reflects loneliness and desolation, capturing the feelings of isolation experienced by characters.'},
  {'name': 'Despair',
   'quote': 'The sun and wind had changed her, too. They had taken the sparkle from her eyes and left them a sober gray; they had taken the red from her cheeks and lips, and they were gray also.',
   'group': 'Emotions',
   'color': '',
   'explanation': ''},
  {'name': 'Childhood Innocence',
   'quote': "When Dorothy, who was an orphan, first came to her, Aunt Em had been so startled by the child's laughter that she would scream and press her hand upon her heart whenever Dorothy's merry voice reached her ears.",
   'group': 'Emotions',
   'color': '',
   'explanation': ''},
  {'name': 'Imminent Danger',
   'quote'

In [2907]:
for name in need_to_add:
    character_info.append({
        "name": name,
        "quote": "",
        "group": "misc",
        "color": "",
        "explanation": ""
    })

len(character_info), character_info

(239,
 [{'name': 'Isolation',
   'quote': 'When Dorothy stood in the doorway and looked around, she could see nothing but the great gray prairie on every side.',
   'group': 'Emotions',
   'color': 'rgb(169, 169, 169)',
   'explanation': 'The gray color reflects loneliness and desolation, capturing the feelings of isolation experienced by characters.'},
  {'name': 'Despair',
   'quote': 'The sun and wind had changed her, too. They had taken the sparkle from her eyes and left them a sober gray; they had taken the red from her cheeks and lips, and they were gray also.',
   'group': 'Emotions',
   'color': '',
   'explanation': ''},
  {'name': 'Childhood Innocence',
   'quote': "When Dorothy, who was an orphan, first came to her, Aunt Em had been so startled by the child's laughter that she would scream and press her hand upon her heart whenever Dorothy's merry voice reached her ears.",
   'group': 'Emotions',
   'color': '',
   'explanation': ''},
  {'name': 'Imminent Danger',
   'quote'

## produce location summaries

In [2908]:
# Pydantic
class LocationInfo(BaseModel):
    """Get information about a location in the story."""
    name: str = Field(description="Name of location.")
    quote: str = Field(description="Direct quote from the story that describes this location")
    emoji: str = Field(description="Emoji that represents this location (e.g., 🏰). Make sure each location has a different emoji.")

class LocationList(BaseModel):
    """Get overall information about the story."""
    locationList: list[LocationInfo] = Field(description="List of location details. One entry per location.")

locationList_llm = llm.with_structured_output(LocationList)

In [2909]:
cropped_json = json.dumps(all_json)[:126000]
location_prompt = f"locations: {all_locations_new}\nstory info: {cropped_json}\nyour output should contain exactly the same {len(all_locations_new)} locations as in the original list."
invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    try:
        res = locationList_llm.invoke(location_prompt)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [2910]:
location_info = []
for l in res.locationList:
    location_info.append({
        "name": l.name,
        "quote": l.quote,
        "emoji": l.emoji
    })

print(len(location_info))
print(location_info)

52
[{'name': 'Kansas', 'quote': "Dorothy lived in the midst of the great Kansas prairies, with Uncle Henry, who was a farmer, and Aunt Em, who was the farmer's wife.", 'emoji': '🌾'}, {'name': "Dorothy's House", 'quote': 'When Dorothy stood in the doorway and looked around, she could see nothing but the great gray prairie on every side.', 'emoji': '🏡'}, {'name': 'Munchkinland', 'quote': 'You are welcome, most noble Sorceress, to the land of the Munchkins. We are so grateful to you for having killed the Wicked Witch of the East.', 'emoji': '👩\u200d🌾'}, {'name': 'Land Of Oz', 'quote': 'Now they are all set free, and are grateful to you for the favor.', 'emoji': '🌈'}, {'name': "Witch's Domain", 'quote': 'Oh, no, that is a great mistake. There were only four witches in all the Land of Oz, and two of them, those who live in the North and the South, are good witches.', 'emoji': '🔮'}, {'name': 'Yellow Brick Road', 'quote': 'The road to the City of Emeralds is paved with yellow brick, so you ca

## generating the final json file!

In [2911]:
# add character and location info to all_json
all_json["characters"] = character_info
all_json["locations"] = location_info
all_json

{'title': 'The Wonderful Wizard of Oz',
 'type': 'Book',
 'author': 'L. Frank Baum',
 'year': 1900,
 'url': 'https://www.gutenberg.org/ebooks/55',
 'image': 'https://m.media-amazon.com/images/I/616u3wbjRYL._AC_UF1000,1000_QL80_.jpg',
 'num_chapters': 23,
 'num_scenes': 165,
 'num_characters': 239,
 'num_locations': 52,
 'chapters': [{'chapter': 'Chapter I The Cyclone',
   'scenes': 2,
   'length': 21537,
   'num_lines': 306,
   'summary': "Dorothy's peaceful life on the prairie is disrupted as her family anticipates a cyclone, which ultimately sweeps their house away into a whirlwind of chaos.",
   'importance': 0.85,
   'conflict': 0.55,
   'locations': {'Kansas': 330, "Dorothy's House": 330},
   'characters': {'Isolation': 330,
    'Despair': 165,
    'Childhood Innocence': 165,
    'Imminent Danger': 165,
    'Family Protection': 165,
    "Nature's Fury": 165,
    'Chaos And Turmoil': 165,
    'Transformation And Change': 165,
    'Fear And Loneliness': 165,
    'Safety And Protecti

In [2912]:
# save as json
with open(f"json/{story_name}/final_data.json", "w") as f:
    json.dump(all_json, f, indent=4)